# Traveling Salesman Problem (TSP)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from copy import copy
import matplotlib.image as mpimg
from matplotlib import animation
from IPython.display import HTML

### Fetching and Visualizing Locations

In [ ]:
def generate_locations(locations = 5):
    coordinates = {}
    for ix in range(locations):
        coordinates[ix] = list(np.random.uniform(-100,100,size=2))  
    return coordinates

def plot_locations(coordinates, annotate=True):
    names = []
    x = []
    y = []
    plt.figure(figsize=(16,6))
    for ix, coord in coordinates.items():
        names.append(ix)
        x.append(coord[0])
        y.append(coord[1])
        if annotate:
            plt.annotate(ix, xy=(coord[0], coord[1]), xytext=(20, -20),
                        textcoords='offset points', ha='right', va='bottom',
                        bbox=dict(boxstyle='round,pad=0.5', fc='w', alpha=0.5),
                        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
    plt.scatter(x,y,c='r',marker='o')
            
def get_path(locations):
    path = copy(locations)
    np.random.shuffle(path)
    path.append(path[0])
    return list(path)

def plot_path(locations, path, count, path_in_title=True):
    plot_locations(locations)
    for ix, loc in enumerate(path[:-1]):
        x = [locations[path[ix]][0],locations[path[ix+1]][0]]
        y = [locations[path[ix]][1],locations[path[ix+1]][1]]
        plt.plot(x,y,'c--',lw=1)
    plt.scatter(locations[path[0]][0],locations[path[0]][1], marker='x', c='b')   
    if path_in_title:
        plt.title("Current path: [%s]"%(','.join([str(x) for x in path])))
    else:
        print("Current path: [%s]"%(','.join([str(x) for x in path])))
    plt.savefig(f'tsp_anim/{count}.png')
    plt.close()

### Genetic Functions

In [ ]:
def create_generation(locations, population=100):
    generation = [get_path(locations) for _ in range(population)]
    return generation

def distance_between_locations(l1, l2):
    l1 = locations[l1]
    l2 = locations[l2]
    distance = np.sqrt((l1[0]-l2[0])**2 + (l1[1]-l2[1])**2)
    return distance

def fitness_score(path):
    score = 0
    for ix, loc_id in enumerate(path[:-1]):
        score += distance_between_locations(loc_id, path[ix+1])
    return score

def check_fitness(paths):
    fitness_indicator = []
    for path in paths:
        fitness_indicator.append((path, fitness_score(path)))
    return fitness_indicator

def get_parents(paths, take_best_N=10, take_random_N=5, verbose=False):
    
    # Getting best paths from last generation (Lower is better)
    fit_scores = check_fitness(paths)
    sorted_paths = sorted(fit_scores, key=lambda x: x[1])
    new_generation = [x[0] for x in sorted_paths[:take_best_N]]
    best_path = new_generation[0]
    
    if verbose:
        print(best_path)
        
    # Selecting random paths
    for _ in range(take_random_N):
        ix = np.random.randint(len(paths))
        new_generation.append(paths[ix])
        
    np.random.shuffle(new_generation)
    return new_generation, best_path

def crossover(parent1, parent2):
    list_of_ids_for_parent1 = list(np.random.choice(parent1, replace=False, size=len(parent1)//2))
    child = [-99 for _ in parent1]
    
    for ix in list_of_ids_for_parent1:
        child[ix] = parent1[ix]
    for ix, gene in enumerate(child):
        if gene == -99:
            for gene2 in parent2:
                if gene2 not in child:
                    child[ix] = gene2
                    break
    child[-1] = child[0]
    return child

def get_children(old_generation, children_per_couple=1):
    mid_point = len(old_generation)//2
    next_generation = [] 
    
    for ix, parent in enumerate(old_generation[:mid_point]):
        for _ in range(children_per_couple):
            next_generation.append(crossover(parent, old_generation[-ix-1]))
    return next_generation

In [ ]:
def evolve_to_solve(current_generation, total_generations, select_best_N, select_random_N,
                    children_per_couple, print_every_n_generations, verbose=False):
    best_paths=[]
    fitness_tracking = []
    for i in range(total_generations):
        if verbose and not i % print_every_n_generations and i > 0:
            print("Generation %i "%i, end='')
            print("Best Score: ", fitness_tracking[-1])
            print()
            is_verbose = True
        else:
            is_verbose = False
        breeders, best_guess = get_parents(current_generation, 
                                                            select_best_N=select_best_N, select_random_N=select_random_N, 
                                                            verbose=is_verbose)
        fitness_tracking.append(fitness_score(best_guess))
        current_generation = get_children(breeders, children_per_couple=children_per_couple)
        best_paths.append(best_guess)
    return fitness_tracking, best_guess, best_paths

In [ ]:
def make_fitness_tracking_plot(fitness_tracking):
    """
    Given a list of fitness scores, plot it versus the generation number
    """
    plt.figure(figsize=(16,6))
    plt.plot(range(len(fitness_tracking)), fitness_tracking)
    plt.ylabel("Fitness Score")
    plt.xlabel("Generation")
    plt.title("Fitness Evolution");

### TSP Example for 40 locations

In [ ]:
locations = generate_locations(40)
current_generation = create_generation(list(locations.keys()),population=300)
fitness_tracking, best_guess, best_paths = evolve_to_solve(current_generation, 150, 100, 50, 5, 10, verbose=True)

In [ ]:
make_fitness_tracking_plot(fitness_tracking)

### Gathering Images

In [ ]:
for i,each in enumerate(best_paths):
    plot_path(locations, each, i)

imgs=[]
for i in range(len(best_paths)):
    imgs.append(mpimg.imread(f'tsp_anim/{i}.png'))
len(imgs)

### Creating Animation from multiple images

In [ ]:
def plot_images(img_list):
    def init():
        img.set_data(img_list[0])
        return (img,)

    def animate(i):
        img.set_data(img_list[i])
        return (img,)

    fig = plt.figure(figsize=(20,8))
    ax = fig.gca()
    img = ax.imshow(img_list[0])
    plt.axis('off')
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames=len(img_list), interval=50, blit=True)
    return anim

In [ ]:
# Viewing Animation
HTML(plot_images(imgs).to_html5_video())

In [ ]:
# Controlling Animation
HTML(plot_images(imgs).to_jshtml())

In [ ]:
# Saving animation
plot_images(imgs).save('tsp.mp4');